In [ ]:
## 感情状態の遷移、応答文選択の時に、下の複数行のコメントアウトの部分を使う予定
## -> 発話の数を調節して、ユーザ発話とシステム発話のペアを作る

def get_all_usertalk_data_from_corpus():
    """
    コーパスから、ユーザ発話のデータを全て取得する
    talker text, talk_content text, emotion text, acceptance text
    """
    ## 下で複数行のコメントアウトをしてるコードは、SVRの学習でデータ数が合わなかったら使うかも
    corpus = sqlite3.connect(constructed_corpus_path)
    cur = corpus.cursor()
    scenes = ["cleaning", "exercise", "game", "lunch", "sleep"]
    dialogue_idx = [str(i) for i in range(200)]
    all_usertalk_data = [] # コーパス内の全てのユーザ発話のデータ
    all_talk_data = [] # コーパス内の全ての発話（ユーザとシステム両方）のデータ
    append_ok_idx_list = []
    append_flag = True
    
    t1 = time.time()
    count = 0
    for scene in scenes:
        for idx in dialogue_idx:
            dialogue_name = scene + idx
            # 一般的なインジェクション攻撃はテーブル名を変える攻撃が無いから、?で置き換えるやつが実装されていない（かも）
            # テーブル名を変える攻撃の場合は文字列の長さで防げるので、formatで大丈夫
            # "select * from {}".format(table)にすると、tableをA where '1'='1'にすれば攻撃できるけど
            # 存在するテーブル名の長さよりも長ければ攻撃だと判定できる（len(table) > (存在するテーブル名): 攻撃だと判定）
            # by.師匠
            cur.execute("select * from {}".format(dialogue_name))
            a_dialogue = cur.fetchall() # fetchall() -> 1対話に含まれる全ての発話データを取得している
            for i, talk_data in enumerate(a_dialogue):
                if talk_data[0] == "user" and if talk_data[3] != "NONE":
                    # ユーザの発話データを見ている、受諾度合いラベルがNONEではない（NONEは学習データから除く）
                    all_usertalk_data.append(talk_data) # ユーザ発話のデータをappend
                    append_ok_idx_list.append(count)
                count += 1
            """
            for i, talk_data in enumerate(a_dialogue):
                count += 1
                if talk_data[0] == "user": # ユーザの発話データを見ている場合
                    if talk_data[3] != "NONE":
                        all_usertalk_data.append(talk_data) # ユーザ発話のデータをappend
                    else:
                        all_talk_data.pop()
                        append_flag = False
                if append_flag:
                    if i + 1 < len(a_dialogue):
                        if a_dialogue[i + 1][0] == "system" and talk_data[0] == "system":
                            pass
                        else:
                            append_ok_idx_list.append(count)
                            all_talk_data.append(talk_data)
                    elif talk_data[0] == "user":
                        append_ok_idx_list.append(count)
                        all_talk_data.append(talk_data)
                else:
                    append_flag = True
                    if i + 2 < len(a_dialogue):
                        if a_dialogue[i + 2][0] == "system":
                            append_flag = False
            """
    
    t2 = time.time()
    print("Finished getting all user talks from {}: about {} seconds.".format(constructed_corpus_path, t2 - t1))
    print("in def_usertalks len(append_ok_idx_list) = {}".format(len(append_ok_idx_list)))
    return all_usertalk_data[:], append_ok_idx_list # 下記SVRの訓練のy, Xの特徴ベクトルのキーに該当する